In [1]:
# !pip install pennylane nftopt

In [2]:
# %pip install nftopt

Testing the Amplitude Embedding - Approach 1 implementation in AWS braket Physical device.

In [3]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

In [4]:
n_qubits = 12
depth = 4

We use the AWS braket - Real quantum computer from 

In [5]:
device_arn = "arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1"
dev = qml.device("braket.aws.qubit", wires=n_qubits, device_arn=device_arn, shots=1024)
dev

<BraketAwsQubitDevice device (wires=12, shots=1024) at 0x7fa0d9a7a9e0>

In [6]:
data = np.loadtxt('12_qubit_data.out')
weights = np.loadtxt('12_qubit_weights.out')

In [7]:
print(data)
print(weights)

[0.00975932 0.02195996 0.00954274 ... 0.01582369 0.02101887 0.01189177]
[ 1.34730403e+01  6.08781112e+00  2.23222438e+00  9.41198146e+00
  7.19025136e+00  6.27874028e+00  3.27355972e+00  1.59422817e-02
  6.16736248e+00  3.13499254e+00  1.02795449e+01  3.14328155e+00
  3.13860280e+00  3.11840574e+00  1.17668482e-02  9.42731269e+00
  3.22188528e+00  6.28057554e+00  4.31305869e+00  3.14920044e+00
  1.12584161e+01  3.11340870e+00  6.77459811e+00  1.25607068e+01
  2.46688751e+00  4.56249561e+00  5.60919706e+00  4.69490231e+00
  2.40140690e+00  7.85524717e+00  9.18206385e+00  1.57091538e+00
  3.63954967e-01  4.70750144e+00  2.93105472e+00  7.85840491e+00
  9.42815699e+00 -2.74568408e-02  9.42519845e+00  3.14230608e+00
  3.13708694e+00  9.42858711e+00  1.25697236e+01  9.41732257e+00
  1.25687459e+01  9.39739819e+00  1.25707670e+01  6.28472098e+00]


In [8]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
        for wire in range(n_qubits):
            qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

        for i in range(n_qubits-1):
            qml.CZ(wires=[i,i+1])
        qml.CZ(wires=[n_qubits-1,0])

In [9]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [10]:
@qml.qnode(dev)
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
  # return qml.state() - for simulator

In [11]:
def variational_classifier(weights):
    # weights are thetas
    return np.real(circuit(weights))

In [ ]:
predictions=variational_classifier(weights)
acc = accuracy(data, predictions)
print(predictions.numpy(), data.numpy())
print(acc)